In [50]:
# Cell 0: Unzip saved_models.zip
import os
import zipfile

print("=" * 70)
print("EXTRACTING SAVED MODELS")
print("=" * 70)

# Check if saved_models.zip exists
if not os.path.exists('saved_models.zip'):
    print("\n❌ ERROR: 'saved_models.zip' not found!")
    print("\nPlease upload 'saved_models.zip' to this notebook directory.")
    print("\nThe zip file should contain:")
    print("  saved_models/")
    print("  ├── inference_utils.py")
    print("  ├── data/")
    print("  │   └── vocab.txt")
    print("  └── models/")
    print("      ├── tfidf_vectorizer_combined.pkl")
    print("      ├── svm_combined.pkl")
    print("      └── label_encoder.pkl")
    raise FileNotFoundError("saved_models.zip not found")

# Extract the zip file
print("\n📦 Extracting saved_models.zip...")
with zipfile.ZipFile('saved_models.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

print("✅ Extraction complete!")

# Verify structure
print("\n🔍 Verifying folder structure...")
expected_files = [
    'saved_models/inference_utils.py',
    'saved_models/data/vocab.txt',
    'saved_models/models/tfidf_vectorizer_combined.pkl',
    'saved_models/models/svm_combined.pkl',
    'saved_models/models/label_encoder.pkl'
]

all_found = True
for file_path in expected_files:
    if os.path.exists(file_path):
        print(f"  ✅ {file_path}")
    else:
        print(f"  ❌ {file_path} - MISSING!")
        all_found = False

if all_found:
    print("\n" + "=" * 70)
    print("✅ ALL FILES EXTRACTED SUCCESSFULLY")
    print("=" * 70)
    print("\n👉 Proceed to Cell 1 to install libraries")
else:
    print("\n" + "=" * 70)
    print("⚠️ WARNING: Some files are missing!")
    print("=" * 70)
    print("\nPlease check your saved_models.zip structure")

EXTRACTING SAVED MODELS

📦 Extracting saved_models.zip...
✅ Extraction complete!

🔍 Verifying folder structure...
  ✅ saved_models/inference_utils.py
  ✅ saved_models/data/vocab.txt
  ✅ saved_models/models/tfidf_vectorizer_combined.pkl
  ✅ saved_models/models/svm_combined.pkl
  ✅ saved_models/models/label_encoder.pkl

✅ ALL FILES EXTRACTED SUCCESSFULLY

👉 Proceed to Cell 1 to install libraries


# cell 1

In [51]:
# Cell 1: Install Required Libraries
print("📦 Installing required libraries...")
print("This may take 2-3 minutes on first run...\n")

!pip install -q transformers torch sentence-transformers rapidfuzz

print("\n✅ All libraries installed successfully!")
print("\nInstalled:")
print("  • transformers (for BERT)")
print("  • torch (PyTorch backend)")
print("  • sentence-transformers (for semantic matching)")
print("  • rapidfuzz (for fuzzy string matching)")

📦 Installing required libraries...
This may take 2-3 minutes on first run...


✅ All libraries installed successfully!

Installed:
  • transformers (for BERT)
  • torch (PyTorch backend)
  • sentence-transformers (for semantic matching)
  • rapidfuzz (for fuzzy string matching)


# cell 2

In [52]:
# Cell 2: Import Libraries and Download NLTK Data
import os
import sys
import pickle
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel, pipeline
from sentence_transformers import SentenceTransformer
import nltk

print("=" * 70)
print("IMPORTING LIBRARIES & DOWNLOADING NLTK DATA")
print("=" * 70)

# Download NLTK data
print("\n📥 Downloading NLTK data...")
try:
    nltk.data.find('tokenizers/punkt')
    nltk.data.find('corpora/stopwords')
    nltk.data.find('corpora/wordnet')
    nltk.data.find('tokenizers/punkt_tab')
    print("✅ NLTK data already present")
except LookupError:
    print("Downloading required NLTK data...")
    nltk.download('punkt', quiet=True)
    nltk.download('stopwords', quiet=True)
    nltk.download('wordnet', quiet=True)
    nltk.download('omw-1.4', quiet=True)
    nltk.download('punkt_tab', quiet=True)
    print("✅ NLTK data downloaded")

print("\n✅ All imports successful!")

# Initialize stop_words for use by inference_utils functions
print("\n🔧 Initializing stop_words...")
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
print(f"   ✅ stop_words initialized ({len(stop_words)} words)")

IMPORTING LIBRARIES & DOWNLOADING NLTK DATA

📥 Downloading NLTK data...
✅ NLTK data downloaded

✅ All imports successful!

🔧 Initializing stop_words...
   ✅ stop_words initialized (198 words)


# cell 3

In [53]:
# Cell 3: Load HuggingFace Models
print("=" * 70)
print("LOADING HUGGINGFACE MODELS")
print("=" * 70)
print("\n⚠️ First time will download models (~1.5GB total)")
print("Subsequent runs will use cached versions\n")

# 1. BERT for semantic similarity
print("1️⃣ Loading BERT (bert-base-uncased)...")
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
bert_model = AutoModel.from_pretrained('bert-base-uncased')
bert_model.eval()
print("   ✅ BERT loaded")

# 2. JobBERT for ML skill extraction
print("\n2️⃣ Loading JobBERT (jjzha/jobbert_knowledge_extraction)...")
ml_knowledge_classifier = pipeline(
    model="jjzha/jobbert_knowledge_extraction",
    aggregation_strategy="first"
)
print("   ✅ JobBERT loaded")

# 3. Sentence Transformer for semantic skill matching
print("\n3️⃣ Loading Sentence Transformer (all-MiniLM-L6-v2)...")
skill_similarity_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
print("   ✅ Sentence Transformer loaded")

print("\n" + "=" * 70)
print("✅ ALL HUGGINGFACE MODELS LOADED")
print("=" * 70)

LOADING HUGGINGFACE MODELS

⚠️ First time will download models (~1.5GB total)
Subsequent runs will use cached versions

1️⃣ Loading BERT (bert-base-uncased)...
   ✅ BERT loaded

2️⃣ Loading JobBERT (jjzha/jobbert_knowledge_extraction)...


Device set to use cuda:0


   ✅ JobBERT loaded

3️⃣ Loading Sentence Transformer (all-MiniLM-L6-v2)...
   ✅ Sentence Transformer loaded

✅ ALL HUGGINGFACE MODELS LOADED


# cell 4

In [54]:
# Cell 4: Load Saved/Trained Models
print("=" * 70)
print("LOADING SAVED MODELS")
print("=" * 70)

# Check if saved_models folder exists
if not os.path.exists('saved_models'):
    print("\n❌ ERROR: 'saved_models/' folder not found!")
    print("Please ensure the following structure exists:")
    print("  saved_models/")
    print("  ├── inference_utils.py")
    print("  ├── data/")
    print("  │   └── vocab.txt")
    print("  └── models/")
    print("      ├── tfidf_vectorizer_combined.pkl")
    print("      ├── svm_combined.pkl")
    print("      └── label_encoder.pkl")
    raise FileNotFoundError("saved_models folder not found")

print("\n📦 Loading trained models from pickle files...\n")

# 1. TF-IDF Vectorizer
print("1️⃣ Loading TF-IDF Vectorizer...")
with open('saved_models/models/tfidf_vectorizer_combined.pkl', 'rb') as f:
    tfidf_vectorizer_combined = pickle.load(f)
print(f"   ✅ Loaded (vocabulary size: {len(tfidf_vectorizer_combined.vocabulary_):,})")

# 2. LinearSVM Classifier
print("\n2️⃣ Loading LinearSVM Classifier...")
with open('saved_models/models/svm_combined.pkl', 'rb') as f:
    svm_combined = pickle.load(f)
print("   ✅ Loaded")

# 3. Label Encoder
print("\n3️⃣ Loading Label Encoder...")
with open('saved_models/models/label_encoder.pkl', 'rb') as f:
    label_encoder = pickle.load(f)
print(f"   ✅ Loaded ({len(label_encoder.classes_)} categories)")
print(f"   Categories: {list(label_encoder.classes_)}")

# 4. Verify vocab.txt exists
print("\n4️⃣ Checking vocab.txt...")
if os.path.exists('saved_models/data/vocab.txt'):
    with open('saved_models/data/vocab.txt', 'r') as f:
        vocab_lines = len(f.readlines())
    print(f"   ✅ Found ({vocab_lines:,} terms)")
else:
    print("   ⚠️ vocab.txt not found (but may not be critical)")

print("\n" + "=" * 70)
print("✅ ALL SAVED MODELS LOADED")
print("=" * 70)

LOADING SAVED MODELS

📦 Loading trained models from pickle files...

1️⃣ Loading TF-IDF Vectorizer...
   ✅ Loaded (vocabulary size: 1,025)

2️⃣ Loading LinearSVM Classifier...
   ✅ Loaded

3️⃣ Loading Label Encoder...
   ✅ Loaded (25 categories)
   Categories: ['Advocate', 'Arts', 'Automation Testing', 'Blockchain', 'Business Analyst', 'Civil Engineer', 'Data Science', 'Database', 'DevOps Engineer', 'DotNet Developer', 'ETL Developer', 'Electrical Engineering', 'HR', 'Hadoop', 'Health and fitness', 'Java Developer', 'Mechanical Engineer', 'Network Security Engineer', 'Operations Manager', 'PMO', 'Python Developer', 'SAP Developer', 'Sales', 'Testing', 'Web Designing']

4️⃣ Checking vocab.txt...
   ✅ Found (934 terms)

✅ ALL SAVED MODELS LOADED


# cell 5

In [55]:
# Cell 5: Import Utility Functions
print("=" * 70)
print("IMPORTING UTILITY FUNCTIONS")
print("=" * 70)

# Add saved_models to Python path
sys.path.append('saved_models')

print("\n📥 Importing from inference_utils.py...\n")

try:
    from inference_utils import (
        # Preprocessing
        clean_resume_text,
        expand_contractions,
        preserve_ngrams,
        preprocess_for_tfidf_enhanced,
        preprocess_for_bert,
        preprocess_for_svm,
        process_new_text,

        # Skill extraction
        extract_skills_and_tools,
        extract_skills_with_ml,

        # Matching
        fuzzy_match_skills,
        ml_semantic_skill_matching,
        get_bert_embedding,
        calculate_bert_similarity,
        match_resume_to_job_with_ml,

        # Display
        format_text_readable,

        # Constants
        contractions_dict,
        tech_ngrams,
    )

    print("✅ Preprocessing functions loaded")
    print("✅ Skill extraction functions loaded")
    print("✅ Matching functions loaded")
    print("✅ Display helper loaded")
    print("✅ Constants loaded")

    # FIX: Inject stop_words into inference_utils module namespace
    import inference_utils
    if not hasattr(inference_utils, 'stop_words'):
        print("\n🔧 Injecting stop_words into inference_utils...")
        from nltk.corpus import stopwords
        inference_utils.stop_words = set(stopwords.words('english'))
        print(f"   ✅ stop_words injected ({len(inference_utils.stop_words)} words)")

except ImportError as e:
    print(f"❌ ERROR importing functions: {e}")
    print("\nMake sure 'saved_models/inference_utils.py' exists and is valid")
    raise

print("\n" + "=" * 70)
print("✅ ALL FUNCTIONS IMPORTED")
print("=" * 70)
print("\n🎉 Setup complete! Ready for inference.")

IMPORTING UTILITY FUNCTIONS

📥 Importing from inference_utils.py...

✅ Preprocessing functions loaded
✅ Skill extraction functions loaded
✅ Matching functions loaded
✅ Display helper loaded
✅ Constants loaded

✅ ALL FUNCTIONS IMPORTED

🎉 Setup complete! Ready for inference.


In [56]:
# Cell 5b: Inject ALL Required Imports and Models into inference_utils
print("=" * 70)
print("INJECTING MODELS AND IMPORTS INTO INFERENCE_UTILS MODULE")
print("=" * 70)

import inference_utils
from nltk.corpus import stopwords
from sentence_transformers import util  # MISSING IMPORT!

print("\n🔧 Injecting all required variables...\n")

# ============================================================================
# 1. NLTK Components
# ============================================================================
print("1️⃣ Injecting NLTK components...")
inference_utils.stop_words = set(stopwords.words('english'))
print(f"   ✅ stop_words ({len(inference_utils.stop_words)} words)")

# ============================================================================
# 2. Sentence Transformers util (for cos_sim)
# ============================================================================
print("\n2️⃣ Injecting sentence_transformers.util...")
inference_utils.util = util
print("   ✅ util (for cosine similarity)")

# ============================================================================
# 3. HuggingFace Models
# ============================================================================
print("\n3️⃣ Injecting HuggingFace models...")
inference_utils.tokenizer = tokenizer
print("   ✅ tokenizer")

inference_utils.bert_model = bert_model
print("   ✅ bert_model")

inference_utils.ml_knowledge_classifier = ml_knowledge_classifier
print("   ✅ ml_knowledge_classifier")

inference_utils.skill_similarity_model = skill_similarity_model
print("   ✅ skill_similarity_model")

# ============================================================================
# 4. Trained Models (from pickle files)
# ============================================================================
print("\n4️⃣ Injecting trained models...")
inference_utils.tfidf_vectorizer_combined = tfidf_vectorizer_combined
print("   ✅ tfidf_vectorizer_combined")

inference_utils.svm_combined = svm_combined
print("   ✅ svm_combined")

inference_utils.label_encoder = label_encoder
print("   ✅ label_encoder")

# ============================================================================
# Verification
# ============================================================================
print("\n" + "=" * 70)
print("VERIFICATION")
print("=" * 70)

required_attrs = [
    'stop_words', 'util', 'tokenizer', 'bert_model',
    'ml_knowledge_classifier', 'skill_similarity_model',
    'tfidf_vectorizer_combined', 'svm_combined', 'label_encoder'
]

all_present = True
for attr in required_attrs:
    if hasattr(inference_utils, attr):
        print(f"  ✅ {attr}")
    else:
        print(f"  ❌ {attr} - MISSING!")
        all_present = False

if all_present:
    print("\n" + "=" * 70)
    print("✅ ALL MODELS AND IMPORTS INJECTED SUCCESSFULLY")
    print("=" * 70)
    print("\n🎉 inference_utils functions can now access everything they need!")
else:
    print("\n⚠️ WARNING: Some required attributes are missing!")

INJECTING MODELS AND IMPORTS INTO INFERENCE_UTILS MODULE

🔧 Injecting all required variables...

1️⃣ Injecting NLTK components...
   ✅ stop_words (198 words)

2️⃣ Injecting sentence_transformers.util...
   ✅ util (for cosine similarity)

3️⃣ Injecting HuggingFace models...
   ✅ tokenizer
   ✅ bert_model
   ✅ ml_knowledge_classifier
   ✅ skill_similarity_model

4️⃣ Injecting trained models...
   ✅ tfidf_vectorizer_combined
   ✅ svm_combined
   ✅ label_encoder

VERIFICATION
  ✅ stop_words
  ✅ util
  ✅ tokenizer
  ✅ bert_model
  ✅ ml_knowledge_classifier
  ✅ skill_similarity_model
  ✅ tfidf_vectorizer_combined
  ✅ svm_combined
  ✅ label_encoder

✅ ALL MODELS AND IMPORTS INJECTED SUCCESSFULLY

🎉 inference_utils functions can now access everything they need!


In [57]:
# Cell 5c: Fix ML Extraction to Handle 512 Token Limit Properly
print("=" * 70)
print("PATCHING ML EXTRACTION FUNCTIONS FOR TOKEN LIMITS")
print("=" * 70)

print("\n🔧 Replacing extract_skills_from_chunk with safer version...")

# Define the fixed version
def extract_skills_from_chunk_fixed(chunk, confidence_threshold=0.5):
    """
    Extract TECHNICAL skills from a single chunk of text
    FIXED: Properly handles 512 token limit
    """
    if not isinstance(chunk, str) or len(chunk) == 0:
        return []

    try:
        # SAFETY: Truncate chunk to ~400 tokens (~1600 chars) to stay WELL under 512 limit
        MAX_CHUNK_CHARS = 1600
        if len(chunk) > MAX_CHUNK_CHARS:
            # Try to break at sentence
            chunk_truncated = chunk[:MAX_CHUNK_CHARS]
            last_period = chunk_truncated.rfind('.')
            if last_period > MAX_CHUNK_CHARS * 0.8:  # If period is in last 20%
                chunk = chunk_truncated[:last_period + 1]
            else:
                chunk = chunk_truncated

        # Extract ONLY Knowledge (technical skills)
        output_knowledge = inference_utils.ml_knowledge_classifier(chunk)

        # Aggregate multi-token spans
        if len(output_knowledge) > 0:
            output_knowledge = inference_utils.aggregate_span(output_knowledge)

        # Filter by confidence and quality
        chunk_skills = []
        for result in output_knowledge:
            if result.get('score', 0) >= confidence_threshold:
                skill_text = result['word'].strip().lower()
                if inference_utils.is_valid_skill(skill_text):
                    chunk_skills.append(skill_text)

        return chunk_skills

    except Exception as e:
        print(f"⚠️ Error processing chunk ({len(chunk)} chars): {str(e)[:100]}")
        return []

# Replace the function in inference_utils
inference_utils.extract_skills_from_chunk = extract_skills_from_chunk_fixed
print("   ✅ extract_skills_from_chunk patched")

# Also fix the sliding window to be more conservative
print("\n🔧 Patching extract_skills_with_ml with more conservative window...")

def extract_skills_with_ml_fixed(text, confidence_threshold=0.6):
    """
    FIXED: More conservative sliding window (1600 chars instead of 2000)
    """
    if not isinstance(text, str) or len(text) == 0:
        return []

    # FIXED: More conservative configuration
    MAX_CHARS = 1600  # ~400 tokens per chunk (was 2000)
    OVERLAP_CHARS = 400  # Overlap (was 500)

    all_skills = set()

    # If text is short enough, process in one pass
    if len(text) <= MAX_CHARS:
        return extract_skills_from_chunk_fixed(text, confidence_threshold)

    # Otherwise, use sliding window
    position = 0
    chunk_num = 0

    while position < len(text):
        chunk_num += 1

        # Extract chunk
        end_position = min(position + MAX_CHARS, len(text))
        chunk = text[position:end_position]

        # If not at end, try to break at sentence boundary
        if end_position < len(text):
            sentence_endings = ['.', '!', '?', '\n']
            best_break = -1

            for i in range(len(chunk) - 1, max(0, len(chunk) - 200), -1):
                if chunk[i] in sentence_endings:
                    best_break = i + 1
                    break

            if best_break > 0:
                chunk = chunk[:best_break]
                end_position = position + best_break

        # Extract skills from this chunk
        chunk_skills = extract_skills_from_chunk_fixed(chunk, confidence_threshold)
        all_skills.update(chunk_skills)

        # Move window forward
        if end_position >= len(text):
            break

        # Move forward with overlap
        position = end_position - OVERLAP_CHARS

        # Safety check to prevent infinite loop
        if position <= chunk_num * 100:
            position = end_position

    return list(all_skills)

# Replace the function in inference_utils
inference_utils.extract_skills_with_ml = extract_skills_with_ml_fixed
print("   ✅ extract_skills_with_ml patched")

print("\n" + "=" * 70)
print("✅ ML EXTRACTION FUNCTIONS PATCHED")
print("=" * 70)
print("\n📊 Changes:")
print("  • Chunk size: 2000 → 1600 chars (~500 → ~400 tokens)")
print("  • Added safety truncation inside extract_skills_from_chunk")
print("  • More conservative overlap: 500 → 400 chars")
print("\n💡 These chunks will now NEVER exceed 512 tokens!")

PATCHING ML EXTRACTION FUNCTIONS FOR TOKEN LIMITS

🔧 Replacing extract_skills_from_chunk with safer version...
   ✅ extract_skills_from_chunk patched

🔧 Patching extract_skills_with_ml with more conservative window...
   ✅ extract_skills_with_ml patched

✅ ML EXTRACTION FUNCTIONS PATCHED

📊 Changes:
  • Chunk size: 2000 → 1600 chars (~500 → ~400 tokens)
  • Added safety truncation inside extract_skills_from_chunk
  • More conservative overlap: 500 → 400 chars

💡 These chunks will now NEVER exceed 512 tokens!


# cell 6

In [58]:
# Cell 6: Check/Create Uploads Folder and Show Instructions
print("=" * 70)
print("CHECKING UPLOAD FOLDER")
print("=" * 70)

# Check if uploads folder exists
uploads_exists = os.path.exists('uploads')
resume_found = False
job_found = False

if uploads_exists:
    print("\n✅ 'uploads/' folder exists")

    # Check for resume file
    for filename in os.listdir('uploads'):
        if 'resume' in filename.lower() and filename.endswith(('.txt', '.pdf', '.docx')):
            resume_found = True
            print(f"   ✅ Resume file found: {filename}")
        if ('job_description' in filename.lower() or ('job' in filename.lower() and 'resume' not in filename.lower())) \
           and filename.endswith(('.txt', '.pdf', '.docx')):
            job_found = True
            print(f"   ✅ Job description file found: {filename}")

    if not resume_found:
        print("   ⚠️ No resume file found")
    if not job_found:
        print("   ⚠️ No job description file found")
else:
    print("\n📁 Creating 'uploads/' folder...")
    os.makedirs('uploads', exist_ok=True)
    print("   ✅ 'uploads/' folder created")

# Show instructions based on what's missing
if resume_found and job_found:
    print("\n" + "=" * 70)
    print("✅ ALL FILES READY!")
    print("=" * 70)
    print("\n👉 You can proceed to the next cells to run inference.")
    print("   (Or upload new files to re-run with different resume/job)")

elif not resume_found or not job_found:
    print("\n" + "=" * 70)
    print("📋 UPLOAD INSTRUCTIONS")
    print("=" * 70)
    print("\nPlease upload the following to the 'uploads/' folder:")

    if not resume_found:
        print("\n  1️⃣ Resume file:")
        print("     • Name must contain: 'resume'")
        print("     • Examples: resume.txt, John_Smith_Resume.pdf, my_resume.docx")
        print("     • Formats: .txt, .pdf, .docx")
    else:
        print("\n  1️⃣ Resume file: ✅ Already uploaded")

    if not job_found:
        print("\n  2️⃣ Job Description file:")
        print("     • Name must contain: 'job' or 'job_description'")
        print("     • Examples: job_description.txt, Senior_Dev_Job.pdf")
        print("     • Formats: .txt, .pdf, .docx")
    else:
        print("\n  2️⃣ Job Description file: ✅ Already uploaded")

    print("\n" + "=" * 70)
    print("After uploading, run the next cells!")
    print("=" * 70)

PREPARING UPLOAD FOLDER

✅ 'uploads/' folder ready

📋 INSTRUCTIONS

Please upload TWO files to the 'uploads/' folder:

  1️⃣ Resume file:
     • Name: resume.txt (or .pdf, .docx)
     • Contains: The candidate's resume

  2️⃣ Job Description file:
     • Name: job_description.txt (or .pdf, .docx)
     • Contains: The job posting/description

💡 Supported formats: .txt, .pdf, .docx

⚠️ File names must be EXACTLY:
   • resume.txt (or resume.pdf, resume.docx)
   • job_description.txt (or job_description.pdf, job_description.docx)

After uploading, run the next cells to process!


# cell 7

In [59]:
# Cell 7: Load and Display Resume
print("=" * 100)
print("📄 LOADING RESUME")
print("=" * 100)

# Check for any file containing "resume" in uploads folder
resume_file = None
if os.path.exists('uploads'):
    for filename in os.listdir('uploads'):
        if 'resume' in filename.lower() and filename.endswith(('.txt', '.pdf', '.docx')):
            resume_file = os.path.join('uploads', filename)
            break

if not resume_file:
    print("\n❌ ERROR: No resume file found!")
    print("Please upload a file containing 'resume' in the name:")
    print("  • uploads/resume.txt")
    print("  • uploads/John_Smith_Resume.pdf")
    print("  • uploads/software_engineer_resume.docx")
    print("  • etc.")
    raise FileNotFoundError("Resume file not found in uploads/ folder")

# Load resume text
print(f"\n✅ Found: {resume_file}")

if resume_file.endswith('.txt'):
    with open(resume_file, 'r', encoding='utf-8') as f:
        resume_text = f.read()
elif resume_file.endswith('.pdf'):
    # For PDF support, would need PyPDF2 or similar
    print("⚠️ PDF support not yet implemented - please use .txt for now")
    raise NotImplementedError("PDF parsing not implemented")
elif resume_file.endswith('.docx'):
    # For DOCX support, would need python-docx
    print("⚠️ DOCX support not yet implemented - please use .txt for now")
    raise NotImplementedError("DOCX parsing not implemented")

print(f"Length: {len(resume_text):,} characters")
print(f"Word count: {len(resume_text.split())} words")

print("\n" + "-" * 100)
print("RESUME CONTENT:")
print("-" * 100)
print()
print(format_text_readable(resume_text, line_length=100))
print()
print("=" * 100)

📄 LOADING RESUME

✅ Found: uploads/example_swe_resume.txt
Length: 3,662 characters
Word count: 472 words

----------------------------------------------------------------------------------------------------
RESUME CONTENT:
----------------------------------------------------------------------------------------------------

# Sample Resume: Alex Rivera

**Alex Rivera**  
San Francisco, CA | (415) 555-0123 | alex.rivera@email.com | linkedin.com/in/alexrivera-dev |
github.com/alexrivera

## Professional Summary
Dynamic Senior Full-Stack Software Engineer with 7+ years of experience building scalable web
applications and integrating AI/ML features for data-intensive platforms. Proven track record at
leading cross-functional teams to deliver high-impact solutions, optimizing performance under tight
deadlines. Expertise in React, Node.js/Python, cloud architectures, and agile methodologies.
Passionate about ethical AI and user-centric design. Seeking to drive innovation at TechNova
Innovatio

# cell 8

In [60]:
# Cell 8: Load and Display Job Description
print("=" * 100)
print("💼 LOADING JOB DESCRIPTION")
print("=" * 100)

# Check for any file containing "job_description" or "job" in uploads folder
job_file = None
if os.path.exists('uploads'):
    for filename in os.listdir('uploads'):
        # Look for "job_description" or "job" (but not "resume")
        if ('job_description' in filename.lower() or ('job' in filename.lower() and 'resume' not in filename.lower())) \
           and filename.endswith(('.txt', '.pdf', '.docx')):
            job_file = os.path.join('uploads', filename)
            break

if not job_file:
    print("\n❌ ERROR: No job description file found!")
    print("Please upload a file containing 'job_description' or 'job' in the name:")
    print("  • uploads/job_description.txt")
    print("  • uploads/Senior_Developer_Job.pdf")
    print("  • uploads/software_engineer_job_posting.docx")
    print("  • etc.")
    raise FileNotFoundError("Job description file not found in uploads/ folder")

# Load job description text
print(f"\n✅ Found: {job_file}")

if job_file.endswith('.txt'):
    with open(job_file, 'r', encoding='utf-8') as f:
        job_text = f.read()
elif job_file.endswith('.pdf'):
    # For PDF support, would need PyPDF2 or similar
    print("⚠️ PDF support not yet implemented - please use .txt for now")
    raise NotImplementedError("PDF parsing not implemented")
elif job_file.endswith('.docx'):
    # For DOCX support, would need python-docx
    print("⚠️ DOCX support not yet implemented - please use .txt for now")
    raise NotImplementedError("DOCX parsing not implemented")

print(f"Length: {len(job_text):,} characters")
print(f"Word count: {len(job_text.split())} words")

print("\n" + "-" * 100)
print("JOB DESCRIPTION CONTENT:")
print("-" * 100)
print()
print(format_text_readable(job_text, line_length=100))
print()
print("=" * 100)

💼 LOADING JOB DESCRIPTION

✅ Found: uploads/example_swe_job_description.txt
Length: 3,481 characters
Word count: 460 words

----------------------------------------------------------------------------------------------------
JOB DESCRIPTION CONTENT:
----------------------------------------------------------------------------------------------------

# Job Description: Senior Full-Stack Software Engineer

**Position:** Senior Full-Stack Software Engineer  
**Company:** TechNova Innovations (a mid-sized SaaS company specializing in AI-driven analytics
tools)
**Location:** Remote (US-based preferred)  
**Employment Type:** Full-time  
**Salary Range:** $140,000 - $180,000 (depending on experience) + equity and benefits  
**Posted Date:** November 4, 2025  

## About TechNova Innovations
TechNova is at the forefront of AI-powered data analytics, helping enterprises unlock insights from
complex datasets. We're a collaborative team of 150+ engineers, designers, and product experts
building s

# cell 9

In [61]:
# Cell 9: Run Inference Pipeline
print("=" * 100)
print("🔄 RUNNING MATCHING PIPELINE")
print("=" * 100)

print("\n⏳ Processing... This may take 30-60 seconds...\n")

# Run the complete matching pipeline
print("1️⃣ Preprocessing texts...")
print("2️⃣ Extracting skills (rule-based & ML)...")
print("3️⃣ Calculating BERT semantic similarity...")
print("4️⃣ Calculating TF-IDF keyword similarity...")
print("5️⃣ Predicting job categories...")
print("6️⃣ Matching skills (fuzzy & semantic)...")
print("7️⃣ Computing final scores...\n")

# Run inference
result = match_resume_to_job_with_ml(resume_text, job_text)

print("✅ Inference complete!")
print("\n" + "=" * 100)
print("📊 Results ready! Run next cell to view detailed analysis.")
print("=" * 100)

🔄 RUNNING MATCHING PIPELINE

⏳ Processing... This may take 30-60 seconds...

1️⃣ Preprocessing texts...
2️⃣ Extracting skills (rule-based & ML)...
3️⃣ Calculating BERT semantic similarity...
4️⃣ Calculating TF-IDF keyword similarity...
5️⃣ Predicting job categories...
6️⃣ Matching skills (fuzzy & semantic)...
7️⃣ Computing final scores...



Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


✅ Inference complete!

📊 Results ready! Run next cell to view detailed analysis.


# cell 10

In [62]:
# Cell 10: Display Detailed Results
print("=" * 100)
print("📊 RESUME-JOB MATCHING RESULTS")
print("=" * 100)

# Extract components
approach1 = result['approach_1_rule_based']
approach2 = result['approach_2_ml_based']

# ============================================================================
# SHARED COMPONENTS
# ============================================================================
print("\n" + "=" * 100)
print("SHARED COMPONENTS")
print("=" * 100)
print(f"\n  BERT Semantic Similarity:  {result['bert_similarity']:.1%} (40% weight)")
print(f"  TF-IDF Keyword Similarity: {result['tfidf_similarity']:.1%} (25% weight)")
print(f"  Category Match:            {result['category_match']:.1%} (15% weight)")
print(f"\n  Resume Category:    {result['resume_category']}")
print(f"  Job Category:       {result['job_category']}")

# ============================================================================
# SKILLS EXTRACTED FROM RESUME
# ============================================================================
print("\n\n" + "=" * 100)
print("SKILLS EXTRACTED FROM RESUME")
print("=" * 100)

print(f"\n📋 Rule-Based (Fuzzy) - {len(approach1['resume_skills'])} skills:")
if len(approach1['resume_skills']) > 0:
    print(f"   {sorted(approach1['resume_skills'])}")
else:
    print("   (none extracted)")

print(f"\n🧠 ML-Based (Neural) - {len(approach2['resume_skills'])} skills:")
if len(approach2['resume_skills']) <= 30:
    print(f"   {sorted(approach2['resume_skills'])}")
else:
    print(f"   {sorted(approach2['resume_skills'])[:30]}")
    print(f"   ... and {len(approach2['resume_skills']) - 30} more")

# ============================================================================
# SKILLS EXTRACTED FROM JOB DESCRIPTION
# ============================================================================
print("\n\n" + "=" * 100)
print("SKILLS EXTRACTED FROM JOB DESCRIPTION")
print("=" * 100)

print(f"\n📋 Rule-Based (Fuzzy) - {len(approach1['job_skills'])} skills:")
if len(approach1['job_skills']) > 0:
    print(f"   {sorted(approach1['job_skills'])}")
else:
    print("   (none extracted)")

print(f"\n🧠 ML-Based (Neural) - {len(approach2['job_skills'])} skills:")
if len(approach2['job_skills']) <= 30:
    print(f"   {sorted(approach2['job_skills'])}")
else:
    print(f"   {sorted(approach2['job_skills'])[:30]}")
    print(f"   ... and {len(approach2['job_skills']) - 30} more")

# ============================================================================
# MATCHED SKILLS
# ============================================================================
print("\n\n" + "=" * 100)
print("MATCHED SKILLS")
print("=" * 100)

print(f"\n📋 Rule-Based (Fuzzy) - {len(approach1['matched_skills'])} matches:")
if len(approach1['matched_skills']) > 0:
    for match in approach1['matched_skills']:
        print(f"   ✓ '{match['job_requires']}' ↔ '{match['resume_has']}' ({match['similarity']:.0f}% similar)")
else:
    print("   (no matches found)")

print(f"\n🧠 ML-Based (Neural) - {len(approach2['matched_skills'])} matches:")
if len(approach2['matched_skills']) > 0:
    for match in approach2['matched_skills']:
        print(f"   ✓ '{match['job_requires']}' ↔ '{match['resume_has']}' ({match['similarity']:.1f}% similar)")
else:
    print("   (no matches found)")

# ============================================================================
# MISSING SKILLS (GAP ANALYSIS)
# ============================================================================
print("\n\n" + "=" * 100)
print("MISSING SKILLS (GAP ANALYSIS)")
print("=" * 100)

print(f"\n📋 Rule-Based (Fuzzy) - {len(approach1['missing_skills'])} missing:")
if len(approach1['missing_skills']) > 0:
    for skill in sorted(approach1['missing_skills']):
        print(f"   ✗ {skill}")
else:
    print("   ✓ No missing skills!")

print(f"\n🧠 ML-Based (Neural) - {len(approach2['missing_skills'])} missing:")
if len(approach2['missing_skills']) > 0:
    # Show first 20, then indicate more
    missing_sorted = sorted(approach2['missing_skills'])
    if len(missing_sorted) <= 20:
        for skill in missing_sorted:
            print(f"   ✗ {skill}")
    else:
        for skill in missing_sorted[:20]:
            print(f"   ✗ {skill}")
        print(f"   ... and {len(missing_sorted) - 20} more")
else:
    print("   ✓ No missing skills!")

# ============================================================================
# SKILLS COMPONENT SCORE
# ============================================================================
print("\n\n" + "=" * 100)
print("SKILLS COMPONENT SCORE (20% weight)")
print("=" * 100)

print(f"\n📋 Rule-Based (Fuzzy):  {approach1['skill_component_score']:.1%}")
print(f"   Match rate: {approach1['match_rate']:.1%}")
print(f"   ({len(approach1['matched_skills'])}/{len(approach1['job_skills'])} skills matched)")

print(f"\n🧠 ML-Based (Neural):   {approach2['skill_component_score']:.1%}")
print(f"   Match rate: {approach2['match_rate']:.1%}")
print(f"   ({len(approach2['matched_skills'])}/{len(approach2['job_skills'])} skills matched)")

# ============================================================================
# FINAL SCORES
# ============================================================================
print("\n\n" + "=" * 100)
print("FINAL SCORES")
print("=" * 100)

print(f"\n📋 Rule-Based (Fuzzy):  {approach1['final_score']:.1%}")
print(f"   Recommendation: {approach1['recommendation']}")

print(f"\n🧠 ML-Based (Neural):   {approach2['final_score']:.1%}")
print(f"   Recommendation: {approach2['recommendation']}")

# ============================================================================
# COMPARISON SUMMARY
# ============================================================================
print("\n\n" + "=" * 100)
print("COMPARISON SUMMARY")
print("=" * 100)

print(f"\n{'Metric':<40} {'Rule-Based':<15} {'ML-Based':<15} {'Difference'}")
print("-" * 100)
print(f"{'Final Score':<40} {approach1['final_score']:.1%}{'':>8} {approach2['final_score']:.1%}{'':>8} {(approach2['final_score']-approach1['final_score']):+.1%}")
print(f"{'Skills Component (20%)':<40} {approach1['skill_component_score']:.1%}{'':>8} {approach2['skill_component_score']:.1%}{'':>8} {(approach2['skill_component_score']-approach1['skill_component_score']):+.1%}")
print(f"{'Resume Skills Extracted':<40} {len(approach1['resume_skills']):<15} {len(approach2['resume_skills']):<15} {len(approach2['resume_skills'])-len(approach1['resume_skills']):+d}")
print(f"{'Job Skills Extracted':<40} {len(approach1['job_skills']):<15} {len(approach2['job_skills']):<15} {len(approach2['job_skills'])-len(approach1['job_skills']):+d}")
print(f"{'Skills Matched':<40} {len(approach1['matched_skills']):<15} {len(approach2['matched_skills']):<15} {len(approach2['matched_skills'])-len(approach1['matched_skills']):+d}")
print(f"{'Match Rate':<40} {approach1['match_rate']:.1%}{'':>8} {approach2['match_rate']:.1%}{'':>8} {(approach2['match_rate']-approach1['match_rate']):+.1%}")

print("\n" + "=" * 100)
print("✅ ANALYSIS COMPLETE")
print("=" * 100)

📊 RESUME-JOB MATCHING RESULTS

SHARED COMPONENTS

  BERT Semantic Similarity:  96.5% (40% weight)
  TF-IDF Keyword Similarity: 54.8% (25% weight)
  Category Match:            100.0% (15% weight)

  Resume Category:    Java Developer
  Job Category:       Java Developer


SKILLS EXTRACTED FROM RESUME

📋 Rule-Based (Fuzzy) - 21 skills:
   ['aws', 'distributed systems', 'django', 'docker', 'gcp', 'git', 'google cloud', 'javascript', 'jenkins', 'kubernetes', 'machine learning', 'mongodb', 'postgresql', 'python', 'pytorch', 'react', 'redis', 'scikit-learn', 'software engineer', 'tensorflow', 'terraform']

🧠 ML-Based (Neural) - 72 skills:
   ['- dev | github.', 'agile / scrum', 'agile methodologies', 'ai / ml', 'ai ml', 'algorithms', 'apache airflow', 'api', 'aws', 'aws lambda', 'bachelor of science', 'backend', 'best practices', 'capstone', 'cd pipelines', 'ci', 'ci / cd pipelines', 'cloud', 'cloud & devops *', 'cloud architectures', 'computer science', 'd3. js', 'data privacy', 'databases 